In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
from keras.utils import to_categorical
import numpy as np
import os
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import glob
from sklearn.utils import shuffle
from io import StringIO
import warnings
import gc
import tensorflow as tf

Using TensorFlow backend.
C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\numba\errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
warnings.filterwarnings("ignore")

if os.path.exists("../input/birdsong-recognition/test_audio/"):
    n_epochs=200
    fc=None
else:
    n_epochs=50
    fc=2

prob_cutoff = 0.5

prepared_test_str = """row_id,site,audio_id,seconds
bulori/XC128942,bulori/XC128942,bulori/XC128942,5
bulori/XC170988,bulori/XC170988,bulori/XC170988,5
normoc/XC54018,normoc/XC54018,normoc/XC54018,5
normoc/XC62791,normoc/XC62791,normoc/XC62791,5
herthr/XC53784,herthr/XC53784,herthr/XC53784,5
herthr/XC119596,herthr/XC119596,herthr/XC119596,5
brnthr/XC31308,brnthr/XC31308,brnthr/XC31308,5
brnthr/XC53695,brnthr/XC53695,brnthr/XC53695,5
vesspa/XC17095,vesspa/XC17095,vesspa/XC17095,5
vesspa/XC17096,vesspa/XC17096,vesspa/XC17096,5
solsan/XC17025,solsan/XC17025,solsan/XC17025,5
norfli/XC11578,norfli/XC11578,norfli/XC11578,5
lesnig/XC27724,lesnig/XC27724,lesnig/XC27724,5
grycat/XC31058,grycat/XC31058,grycat/XC31058,5
eastow/XC53188,eastow/XC53188,eastow/XC53188,5
aldfly/XC2628,aldfly/XC2628,aldfly/XC2628,5
ameavo/XC99571,ameavo/XC99571,ameavo/XC99571,5
amebit/XC127371,amebit/XC127371,amebit/XC127371,5
amecro/XC51410,amecro/XC51410,amecro/XC51410,5
amegfi/XC17120,amegfi/XC17120,amegfi/XC17120,5
"""

In [3]:
def load_test_audio(path):
    signal, blah = librosa.load(path, sr=22050)
    
    if len(signal.shape)==1:
        y = signal
    else:
        y = np.average(signal, axis=1)
    
    return signal
    

def getBirdMfcc(bird, n_mfcc=50, filecount=None):
    birddirectory = "data/mfcc_50/" + bird + ".pickle"
    mfcc = pickle.load(open(birddirectory,'rb')).T
    return mfcc

In [4]:
mfcc_directory = '../input/mfcc-50/'
birds = ["bulori", "normoc", "herthr", "brnthr", "vesspa",    'aldfly', 'ameavo', 'amebit', 'amecro', 'amegfi']

ohe = OneHotEncoder(sparse=False)
le = LabelEncoder()

features = np.array(birds[:5])
fint = le.fit_transform(features).reshape(len(features),1)

ohe.fit(fint)

OneHotEncoder(categorical_features='all', dtype=<class 'float'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [5]:
dataMap = {}
testMap = {}

for bird in birds:
    bird_mfcc = getBirdMfcc(bird, filecount=fc)
    train_samples = int(0.7*bird_mfcc.shape[0])
    test_samples = -1*int(0.3*bird_mfcc.shape[0])
    dataMap[bird] = bird_mfcc[:train_samples,]
    testMap[bird] = bird_mfcc[test_samples:,]

In [6]:
model = Sequential()

model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(None, 50)))
model.add(Bidirectional(LSTM(25, return_sequences=True)))
model.add(Dense(15, activation = 'tanh'))
model.add(Dense(5, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
def train_generator():
    while True:
        sequence_length = 215
        
        xarr = []
        yarr = []
        for k,v in dataMap.items():
            nsamples = v.shape[0]
            samples = np.random.randint(0,nsamples-sequence_length+1,size=10)
            try:
                birdohe = ohe.transform(le.transform([k]).reshape(1,1))
            except:
                birdohe = np.array([[0, 0, 0, 0, 0]])
            for sample in samples:
                xarr.append(v[sample:sample+sequence_length,:].reshape(1,sequence_length,50))
                yarr.append(np.tile(birdohe,(sequence_length,1)).reshape(1,sequence_length,5))
        x_train = np.concatenate(xarr)
        y_train = np.concatenate(yarr)
        x_train, y_train = shuffle(x_train, y_train)
        yield x_train, y_train

def test_generator():
    sequence_length = 215

    xarr = []
    yarr = []
    for k,v in testMap.items():
        nsamples = v.shape[0]
        samples = np.random.randint(0,nsamples-sequence_length+1,size=1000)
        try:
            birdohe = ohe.transform(le.transform([k]).reshape(1,1))
        except:
            birdohe = np.array([[0, 0, 0, 0, 0]])
        for sample in samples:
            xarr.append(v[sample:sample+sequence_length,:].reshape(1,sequence_length,50))
            yarr.append(np.tile(birdohe,(sequence_length,1)).reshape(1,sequence_length,5))
    x_train = np.concatenate(xarr)
    y_train = np.concatenate(yarr)
    x_train, y_train = shuffle(x_train, y_train)
    return x_train, y_train

x_val,y_val = test_generator()

# x,y = train_generator()
history = model.fit_generator(train_generator(), steps_per_epoch=30, epochs=n_epochs, verbose=1, validation_data=(x_val, y_val))

Epoch 1/50
30/30 [==============================] - 75s 3s/step - loss: 0.4160 - acc: 0.8633 - val_loss: 0.3311 - val_acc: 0.9000
Epoch 2/50
30/30 [==============================] - 69s 2s/step - loss: 0.3150 - acc: 0.9000 - val_loss: 0.3164 - val_acc: 0.9000
Epoch 3/50
30/30 [==============================] - 68s 2s/step - loss: 0.2958 - acc: 0.9007 - val_loss: 0.3119 - val_acc: 0.8982
Epoch 4/50
30/30 [==============================] - 66s 2s/step - loss: 0.2749 - acc: 0.9045 - val_loss: 0.2996 - val_acc: 0.8993
Epoch 5/50
30/30 [==============================] - 65s 2s/step - loss: 0.2594 - acc: 0.9090 - val_loss: 0.2953 - val_acc: 0.9070
Epoch 6/50
30/30 [==============================] - 65s 2s/step - loss: 0.2503 - acc: 0.9116 - val_loss: 0.2887 - val_acc: 0.9035
Epoch 7/50
30/30 [==============================] - 64s 2s/step - loss: 0.2449 - acc: 0.9145 - val_loss: 0.2838 - val_acc: 0.9071
Epoch 8/50
30/30 [==============================] - 65s 2s/step - loss: 0.2369 - acc: 0.91

In [8]:
plt.subplot(2,1,1)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

plt.subplot(2,1,2)
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.show()

In [9]:
if os.path.exists("../input/birdsong-recognition/test_audio/"):
    test_dir = "../input/birdsong-recognition/test_audio/"
    testtable = pd.read_csv("../input/birdsong-recognition/test.csv")
else:
    test_dir = "../input/birdsong-recognition/train_audio/"
    testtable = pd.read_csv(StringIO(prepared_test_str))


def load_test_clip(signal, start_time, duration=5):
    sr=22050
    maxl = len(signal)
    
    if duration!=None:
        y = signal[max(0,int(start_time)*22050):min(int(start_time+duration)*22050,maxl)]
    else:
        y = signal[max(0,start_time*22050):]
    
    mfcc_feat = librosa.feature.mfcc(n_mfcc=50, y=y, sr=22050)
    return mfcc_feat.T

def make_prediction(sound_clip):
    predidx = [i for i,each in enumerate(make_probabilities(sound_clip)) if each>prob_cutoff]
    if len(predidx)==0:
        predbirds = "nocall"
    else:
        predbirds = " ".join(list(le.inverse_transform(predidx)))
    
    return predbirds

def make_probabilities(sound_clip):
    mfccdim = sound_clip.shape
    sound_clip = sound_clip.reshape(1,mfccdim[0],mfccdim[1])
    
    pred = model.predict(sound_clip).reshape(mfccdim[0],5)
    
    return np.mean(pred,axis=0)

In [10]:
test_info = testtable.sort_values("audio_id")

preds = []
sound_read = ""
for index, row in test_info.iterrows():
    # Get test row information
    site = row['site']
    try:
        start_time = row['seconds'] - 5
    except:
        start_time = 0
    row_id = row['row_id']
    audio_id = row['audio_id']

    try:
        if sound_read != audio_id:
            sound_audio = load_test_audio(test_dir + audio_id + '.mp3')
            sound_read = audio_id
    except:
        print("exception")
        pred = "nocall"
        preds.append(pred)
        continue
        
    if site == 'site_3':
        sound_clip = load_test_clip(sound_audio, 0, duration=None)
    else:
        sound_clip = load_test_clip(sound_audio, start_time)
            
    
    pred = make_prediction(sound_clip)
    preds.append(pred)
    
testtable = test_info.sort_index()

test_submission = testtable.drop(['site', 'audio_id', 'seconds'], axis = 1) 
test_submission['birds'] = preds

test_submission.to_csv('submission.csv', index = None)

exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception


In [11]:
print(test_submission.head(20))

             row_id   birds
0   bulori/XC128942  nocall
1   bulori/XC170988  nocall
2    normoc/XC54018  nocall
3    normoc/XC62791  nocall
4    herthr/XC53784  nocall
5   herthr/XC119596  nocall
6    brnthr/XC31308  nocall
7    brnthr/XC53695  nocall
8    vesspa/XC17095  nocall
9    vesspa/XC17096  nocall
10   solsan/XC17025  nocall
11   norfli/XC11578  nocall
12   lesnig/XC27724  nocall
13   grycat/XC31058  nocall
14   eastow/XC53188  nocall
15    aldfly/XC2628  nocall
16   ameavo/XC99571  nocall
17  amebit/XC127371  nocall
18   amecro/XC51410  nocall
19   amegfi/XC17120  nocall
